In [17]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from dateutil.parser import parse
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import ADASYN
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier

from sklearn.linear_model import LogisticRegression


In [25]:
feature_set =  'feature_set_6_normalised'
X_train = pd.read_csv( F'./data/{feature_set}/X_train_full.csv').drop(columns=['TX_ID','Time_of_day'])
y_train = pd.read_csv( F'./data/{feature_set}/y_train.csv').values.ravel()

X_val = pd.read_csv( F'./data/{feature_set}/X_valid_full.csv').drop(columns=['TX_ID','Time_of_day'])
y_val = pd.read_csv( F'./data/{feature_set}/y_valid.csv').values.ravel()

In [22]:
X_train.dtypes

FAILURE_CODE_6                         int64
FAILURE_REASON_0                       int64
FAILURE_REASON_Invalid merchant        int64
FAILURE_CODE_59                        int64
CARD_COUNTRY_CODE_CA                   int64
                                       ...  
FAILURE_CODE_11                        int64
TRANSACTION_CURRENCY_MDL               int64
FAILURE_REASON_Refer to card issuer    int64
FAILURE_CODE_Z1                        int64
FAILURE_CODE_54                        int64
Length: 112, dtype: object

In [26]:
xgboost = XGBClassifier(n_estimators = 1000,random_state=0,n_jobs=-1)
eval_set = [(X_val, y_val)]
xgboost.fit(X_train, y_train, eval_metric="auc", eval_set=eval_set, early_stopping_rounds=100, verbose=True)

c:\Users\joshu\anaconda3\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\joshu\anaconda3\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.86934
[1]	validation_0-auc:0.86991
[2]	validation_0-auc:0.87010
[3]	validation_0-auc:0.87430
[4]	validation_0-auc:0.87498
[5]	validation_0-auc:0.87515
[6]	validation_0-auc:0.87875
[7]	validation_0-auc:0.87895
[8]	validation_0-auc:0.87938
[9]	validation_0-auc:0.88178
[10]	validation_0-auc:0.88172
[11]	validation_0-auc:0.88225
[12]	validation_0-auc:0.88271
[13]	validation_0-auc:0.88484
[14]	validation_0-auc:0.88480
[15]	validation_0-auc:0.88630
[16]	validation_0-auc:0.88653
[17]	validation_0-auc:0.88905
[18]	validation_0-auc:0.88932
[19]	validation_0-auc:0.88958
[20]	validation_0-auc:0.88951
[21]	validation_0-auc:0.88934
[22]	validation_0-auc:0.89041
[23]	validation_0-auc:0.89073
[24]	validation_0-auc:0.89090
[25]	validation_0-auc:0.89125
[26]	validation_0-auc:0.89132
[27]	validation_0-auc:0.89145
[28]	validation_0-auc:0.89220
[29]	validation_0-auc:0.89219
[30]	validation_0-auc:0.89233
[31]	validation_0-auc:0.89246
[32]	validation_0-auc:0.89238
[33]	validation_0-au

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=-1,
              num_parallel_tree=None, random_state=0, ...)

In [27]:

val_res = 1 -xgboost.predict_proba(X_val)[:,0]
roc_auc_score(y_val,val_res)


0.8944315650966936

In [28]:
test_df = pd.read_csv( F'./data/{feature_set}/test_features.csv')

In [31]:
test_res = 1 -xgboost.predict_proba(test_df.drop(columns=['TX_ID','Time_of_day','TX_FRAUD']))[:,0]

In [32]:
test_df['TX_FRAUD']  = test_res

In [33]:
submission_XGB = test_df[['TX_ID','TX_FRAUD']]

In [34]:
test_trans = pd.read_csv( F'./data/transactions_test.csv')
test_trans = test_trans['TX_ID']
submission_XGB.set_index('TX_ID',inplace=True)
submission = submission_XGB.reindex(index =test_trans)

In [35]:
location = 'results'
submission.to_csv(F'./{location}/results_6_XGB.csv',index=True)